In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

import pandas as pd
import openai

### Read the sampled dataset with Amazon inventory data

In [ ]:
df_items = pd.read_json("../../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines=True)

In [ ]:
df_items.head()

In [ ]:
list(df_items["features"].items())[0]

In [ ]:
list(df_items["images"].items())[0]

### Preprocess title and features

In [ ]:
def preprocess_description(row):
    return f"{row['title']} {' '.join(row['features'])}"

In [ ]:
def extract_first_large_image(row):
    return row["images"][0].get("large", "")

In [ ]:
df_items["description"] = df_items.apply(preprocess_description, axis=1)
df_items["image"] = df_items.apply(extract_first_large_image, axis=1)

In [ ]:
df_items.head()

In [ ]:
list(df_items["description"].items())[0]

### Sample 50 items from the dataset

In [ ]:
df_sample = df_items.sample(50, random_state=42)

In [ ]:
len(df_sample)

In [ ]:
data_to_embed = df_sample[["description", "image", "rating_number", "price", "average_rating", "parent_asin"]].to_dict(orient="records")

In [ ]:
data_to_embed

### Define the embedding function

In [ ]:
response = openai.embeddings.create(
    input="Random text",
    model="text-embedding-3-small",
)

In [ ]:
len(response.data[0].embedding)

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    return response.data[0].embedding

In [ ]:
get_embedding("Hi")

### Create Qdrant collection

In [ ]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [ ]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-00",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

### Embed data

##### Test

In [ ]:
pointstruct = PointStruct(
    id=0,
    vector=get_embedding("Test text"),
    payload={
        "text": "Test text",
        "model": "text-embedding-3-small",
    },
)

In [ ]:
pointstruct

### Amazon data

In [ ]:
pointstructs = []
for i, data in enumerate(data_to_embed):
    embediing = get_embedding(data["description"])
    pointstructs.append(
        PointStruct(
            id=i,
            vector=embediing,
            payload=data,
        )
    )

In [ ]:
pointstructs

In [ ]:
len(pointstructs)

### Write embedded data to Qdrant

In [ ]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-00",
    wait=True,
    points=pointstructs,
)

### Define a function for data retrieval

In [ ]:
def retrieve_data(query, k=5):
    query_embedding = get_embedding(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )
    return results

### Test retrieval

In [ ]:
retrieve_data("What kind of charging cords do you offer?", k=10).points